In [20]:
from uuid import uuid4
import pandas as pd
from langchain_core.documents import Document

def load_final_df():
    final_df_path = r"D:\DATN\QA_System\data_analyze\finaldf0.xlsx"
    if os.path.exists(final_df_path):
        final_df = pd.read_excel(final_df_path)
        print(f"Đã load final_df từ file: {final_df_path}")
        return final_df
    else:
        print("File final_df chưa tồn tại.")
        return None
def convert_df_to_documents(final_df):
    documents = []
    for _, row in final_df.iterrows():
        # Handle metadata - convert to dict if it's a string
        metadata = row["metadata"]
        if isinstance(metadata, str):
            source = metadata
        else:
            source = metadata.get("source", "unknown") if isinstance(metadata, dict) else "unknown"

        # Create Document with properly handled metadata
        doc = Document(
            page_content=row["text"],
            metadata={
                "source": source,
                "level": row["level"]
            }
        )
        documents.append(doc)

    # Process documents
    processed_documents = []
    for doc in documents:
        # Handle source field
        if isinstance(doc.metadata["source"], list):
            doc.metadata["source"] = " ".join(doc.metadata["source"]) if doc.metadata["source"] else "..."
        elif doc.metadata["source"] is None:
            doc.metadata["source"] = "..."

        # Skip empty content
        if not doc.page_content.strip():
            print(f"Warning: Page content is empty for document: {doc.metadata['source']}")
            continue

        processed_documents.append(doc)

    print(f"Processed {len(processed_documents)} out of {len(documents)} documents.")
    return processed_documents

final_df = load_final_df()
documents = convert_df_to_documents(final_df)



Đã load final_df từ file: D:\DATN\QA_System\data_analyze\finaldf0.xlsx
Processed 981 out of 981 documents.


In [21]:
documents

[Document(metadata={'source': "{'source': 'Ban giám đốc Đại học.txt', 'id': '0'}", 'level': 0}, page_content='Giám đốc Đại học:\xa0PGS.TS. Huỳnh Quyết Thắng\nEmail: thang.huynhquyet@hust.edu.vn\xa0\nPhó Giám đốc:\xa0PGS.TS. Nguyễn Phong Điền\nEmail: dien.nguyenphong@hust.edu.vn\xa0\nPhó Giám đốc: PGS.TS. Huỳnh Đăng Chính\nEmail: chinh.huynhdang@hust.edu.vn\xa0\nPhó Giám đốc: PGS.TS. Trần Ngọc Khiêm\nEmail: khiem.tranngoc@hust.edu.vn'),
 Document(metadata={'source': "{'source': 'Chiến lược phát triển đại học Bách khoa Hà Nội.txt', 'id': '0'}", 'level': 0}, page_content='CHIẾN LƯỢC PHÁT TRIỂN\nTÓM LƯỢC CHIẾN LƯỢC\xa0PHÁT\xa0TRIỂN ĐẠI HỌC BÁCH KHOA HÀ\xa0NỘI\xa0\nGIAI ĐOẠN 2017-2025\nI. MỤC TIÊU\xa0\nPhát triển thành một đại học nghiên cứu đa lĩnh vực với nòng cốt là kỹ thuật và công nghệ, trong đó các đơn vị chuyên môn được tổ chức thành một số trường và khoa; viện và trung tâm nghiên cứu trực thuộc.\xa0\nXây dựng hình mẫu thành công, phát triển bền vững của một đại học tự chủ toàn diện 

In [22]:
# Chuyển đổi sang JSON
def convert_document_to_json(doc: Document):
    # Giải mã metadata nếu nó là chuỗi JSON
    metadata = doc.metadata
    if isinstance(metadata.get("source"), str):
        try:
            metadata["source"] = json.loads(metadata["source"].replace("'", '"'))
        except json.JSONDecodeError:
            pass  # Nếu không decode được, giữ nguyên

    return {
        "metadata": metadata,
        "page_content": doc.page_content
    }

json_documents = [convert_document_to_json(doc) for doc in documents]

print(json.dumps(json_documents, indent=2, ensure_ascii=False))
from elasticsearch import Elasticsearch, helpers
import json

# Khởi tạo Elasticsearch client

# Cấu hình index và mapping
index_name = "base"
mapping = {
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "metadata": {"type": "object"},
            "vector": {"type": "dense_vector", "dims": 768, "similarity": "cosine", "index": True}  # Số chiều của vector embedding
        }
    }
}

# Kiểm tra nếu index đã tồn tại, xóa và tạo mới
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
es.indices.create(index=index_name, body=mapping)

# Chuyển đổi json_documents thành định dạng Elasticsearch bulk
def generate_actions(documents):
    for record in documents:
        # Tạo vector embedding từ nội dung page_content
        embedding = embeddings.embed_query(record["page_content"])
        
        yield {
            "_op_type": "index",  # Hoặc "create" nếu bạn muốn tránh việc ghi đè
            "_index": index_name,
            "_source": {
                "text": record["page_content"],
                "metadata": record["metadata"],
                "vector": embedding
            }
        }

# Sử dụng helpers.bulk để insert batch dữ liệu vào Elasticsearch
helpers.bulk(es, generate_actions(json_documents))

print(f"Đã chèn {len(json_documents)} bản ghi vào Elasticsearch.")


[
  {
    "metadata": {
      "source": {
        "source": "Ban giám đốc Đại học.txt",
        "id": "0"
      },
      "level": 0
    },
    "page_content": "Giám đốc Đại học: PGS.TS. Huỳnh Quyết Thắng\nEmail: thang.huynhquyet@hust.edu.vn \nPhó Giám đốc: PGS.TS. Nguyễn Phong Điền\nEmail: dien.nguyenphong@hust.edu.vn \nPhó Giám đốc: PGS.TS. Huỳnh Đăng Chính\nEmail: chinh.huynhdang@hust.edu.vn \nPhó Giám đốc: PGS.TS. Trần Ngọc Khiêm\nEmail: khiem.tranngoc@hust.edu.vn"
  },
  {
    "metadata": {
      "source": {
        "source": "Chiến lược phát triển đại học Bách khoa Hà Nội.txt",
        "id": "0"
      },
      "level": 0
    },
    "page_content": "CHIẾN LƯỢC PHÁT TRIỂN\nTÓM LƯỢC CHIẾN LƯỢC PHÁT TRIỂN ĐẠI HỌC BÁCH KHOA HÀ NỘI \nGIAI ĐOẠN 2017-2025\nI. MỤC TIÊU \nPhát triển thành một đại học nghiên cứu đa lĩnh vực với nòng cốt là kỹ thuật và công nghệ, trong đó các đơn vị chuyên môn được tổ chức thành một số trường và khoa; viện và trung tâm nghiên cứu trực thuộc. \nXây dựng hình m

In [42]:
def search(query, top_k=1):
  embedding = embeddings.embed_query(query)
  result = es.search(
		index="base",
    knn={
      "field": "vector",
      "query_vector": embedding,
      "k": top_k,
      "num_candidates": top_k * 2
		},
    source_excludes=["vector"],
    min_score = 0.85
	)
  res = [hit['_source'] for hit in result['hits']['hits']]
  
  return res

In [44]:
a = search("Học phí của ngành Công nghệ thông tin là bao nhiêu?")

In [45]:
a

[{'text': 'Khoa đã góp phần đào tạo ra hàng nghìn kỹ sư các ngành Công nghệ thông tin, Kỹ thuật phần mềm, Hệ thông tin và Khoa học máy tính, trong đó nhiều người đã trở thành giảng viên tại các trường đại học, hoặc nắm vai trò chủ chốt tại các công ty Công nghệ thông tin trong nước hay đang làm việc cho các tập đoàn công nghệ lớn trên thế giới. Khoa cũng là nơi ươm mầm, nuôi dưỡng và là cái nôi khởi nghiệp cho nhiều ý tưởng độc đáo và sáng tạo. Số lượng doanh nghiệp khởi nghiệp và thành công trong lĩnh vực CNTT xuất phát từ sinh viên và cựu sinh viên của Khoa tăng đều hàng năm.\n2. Kết quả nghiên cứu tiêu biểu\nHệ thống ERP cho trường Đại học eUni:',
  'metadata': {'source': {'source': 'Giới thiệu Khoa Khoa học máy tính.txt',
    'id': '14'},
   'level': 0}}]

In [47]:
a[0]['text']

'Khoa đã góp phần đào tạo ra hàng nghìn kỹ sư các ngành Công nghệ thông tin, Kỹ thuật phần mềm, Hệ thông tin và Khoa học máy tính, trong đó nhiều người đã trở thành giảng viên tại các trường đại học, hoặc nắm vai trò chủ chốt tại các công ty Công nghệ thông tin trong nước hay đang làm việc cho các tập đoàn công nghệ lớn trên thế giới. Khoa cũng là nơi ươm mầm, nuôi dưỡng và là cái nôi khởi nghiệp cho nhiều ý tưởng độc đáo và sáng tạo. Số lượng doanh nghiệp khởi nghiệp và thành công trong lĩnh vực CNTT xuất phát từ sinh viên và cựu sinh viên của Khoa tăng đều hàng năm.\n2. Kết quả nghiên cứu tiêu biểu\nHệ thống ERP cho trường Đại học eUni:'

In [7]:
load_dotenv()
api_key = os.getenv("api_key")
ELASTIC_URL = os.getenv("ELASTIC_URL")

In [5]:
from langchain_elasticsearch import ElasticsearchRetriever
from typing import Dict
def vector_query(search_query: str) -> Dict:
    vector = embeddings.embed_query(search_query)  # same embeddings as for indexing
    return {
        "knn": {
            "field": "vector",
            "query_vector": vector,
            "k": 1,
            "num_candidates": 10,
        },
        "min_score": 0.85,
       # "source_excludes":["vector"],
    }
# def search(query, top_k=1):
#   embedding = embeddings.embed_query(query)
#   result = es.search(
# 		index="base",
#     knn={
#       "field": "vector",
#       "query_vector": embedding,
#       "k": top_k,
#       "num_candidates": top_k * 2
# 		},
#     source_excludes=["vector"],
#     min_score = 0.85
# 	)
#   res = [hit['_source'] for hit in result['hits']['hits']]
  
#   return res

def qaretrieval(index_name: str) -> ElasticsearchRetriever:
    return  ElasticsearchRetriever.from_es_params(
    index_name=index_name,
    body_func=vector_query,
    content_field="answer",
    url=  ELASTIC_URL,
    api_key = api_key
)
    


In [8]:
def hybrid_query(search_query: str, k_text: int = 50, k_vector: int = 50, k_final: int = 5) -> Dict:

    # Chuyển câu hỏi thành vector embedding
    vector = embeddings.embed_query(search_query)
    
    return {
        "retriever": {
            "rrf": {
                "retrievers": [
                    {
                        "standard": {
                            "query": {
                                "match": {
                                    "text": search_query,
                                }
                            }
                        }
                    },
                    {
                        "knn": {
                            "field": "vector",
                            "query_vector": vector,
                            "k": 50,  # Vector search: tìm kiếm 50 kết quả
                            "num_candidates": 50 * 2  # Số ứng viên xét là gấp đôi k_vector
                        }
                    }
                    
                ],
            "rank_window_size": 50,
            "rank_constant": 60 #default = 60
            }
        }, "size":5
  
        
    }

def retrieval(index_name: str) -> ElasticsearchRetriever:
    return  ElasticsearchRetriever.from_es_params(
    index_name=index_name,
    body_func=hybrid_query,
    content_field="text",
    url=  ELASTIC_URL,
    api_key = api_key
)
hybrid_search = retrieval("raptor")


In [12]:
query= "giám đốc đại học bách khoa là ai"

In [18]:
result = hybrid_search.invoke(query)
result

[Document(metadata={'_index': 'raptor', '_id': 'Ef8fJZQBZ5S_PGgdPFE4', '_score': 0.032002047, '_source': {'metadata': {'source': {'source': 'Đảng ủy Trường CNTT&TT.txt', 'id': '2'}, 'level': 0}, 'vector': [-0.02527233399450779, -0.031692445278167725, -0.09841200709342957, 0.12007340788841248, -0.05693375691771507, 0.3893887996673584, 0.3633063733577728, 0.2748876214027405, 0.20495109260082245, -0.1754806786775589, 0.10762425512075424, -0.09326538443565369, 0.433308482170105, -0.05625097081065178, 0.08048601448535919, -0.1499662846326828, 0.25246044993400574, 0.30933186411857605, -0.2018660008907318, -0.16553598642349243, -0.07603208720684052, -0.05782054364681244, 0.2783011496067047, 0.19564008712768555, 0.07910417020320892, 0.27177736163139343, 0.24092994630336761, 0.05499961972236633, 0.022616809234023094, 0.030868206173181534, 0.29692044854164124, 0.013778205029666424, -0.07798027247190475, -0.24118511378765106, 0.1780463457107544, -0.1333361715078354, 0.17859552800655365, 0.0415542

In [24]:
res = [doc.page_content for doc in result]
res

['Bí thư:\xa0TS. Lê Xuân Thành\nHội đồng Trường CNTT&TT:\xa0là cơ quan do Giám đốc Đại học Bách khoa Hà Nội ra quyết định thành lập để tư vấn cho Giám đốc trong việc triển khai thực hiện các nhiệm vụ quản lý và điều hành Đại học.\nHội đồng Trường nhiệm kỳ 2022 – 2027 gồm 15 thành viên, trong đó có 03 thành viên ngoài Trường là các nhà khoa học, nhà quản lý, và chuyên gia công nghệ uy tín trong và ngoài nước.\nDanh sách thành viên Hội đồng Trường CNTT&TT nhiệm kỳ 2022 – 2027',
 'Cụm văn bản này cung cấp thông tin liên hệ của các lãnh đạo cấp cao của Đại học Bách Khoa Hà Nội, bao gồm Giám đốc và ba Phó Giám đốc.  Cụ thể,  đó là tên, chức danh và địa chỉ email của PGS.TS. Huỳnh Quyết Thắng (Giám đốc), PGS.TS. Nguyễn Phong Điền, PGS.TS. Huỳnh Đăng Chính và PGS.TS. Trần Ngọc Khiêm (ba Phó Giám đốc).\n',
 'Giới\xa0thiệu chung:\nTên tiếng Anh: the International Research Center for Artificial Intelligence (BK.AI)\nGiám đốc khoa học:\xa0GS. Hồ Tú Bảo\nGiám đốc điều hành: \xa0TS. Nguyễn Phi Lê\n

In [ ]:
def semantic_query(search_query: str, top_k = 5) -> Dict:
    vector = embeddings.embed_query(search_query)  # same embeddings as for indexing
    return {
        "knn": {
            "field": "vector",
            "query_vector": vector,
            "k": top_k,
            "num_candidates": top_k * 2, # default = 10
        },
        #"min_score": 0.85,
        "source_excludes":["vector"],
    }
def semantic_retriever(index_name: str) -> ElasticsearchRetriever:
    return  ElasticsearchRetriever.from_es_params(
    index_name=index_name,
    body_func=semantic_query,
    content_field="text",
    url=  ELASTIC_URL,
    api_key = api_key
    )